In [10]:
#!/usr/bin/env python3
"""
sift_match_baseline.py
Single-file SIFT matching pipeline with RANSAC-based homography filtering.
Runs in both Python script and Jupyter environments.
"""

import cv2
import numpy as np
import os
from pathlib import Path
import csv
import sys

# ---------- Configuration ----------
IMG1_PATH = Path("images/img1.jpg")
IMG2_PATH = Path("images/img2.jpg")
RESULTS_DIR = Path("results")
RATIO_TEST = 0.75
RANSAC_REPROJ_THRESH = 4.0
MAX_MATCHES_DRAW = 200
# -----------------------------------

def ensure_dirs():
    Path("images").mkdir(parents=True, exist_ok=True)
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def debug_print_paths():
    print("Working directory:", Path.cwd())
    # handle Jupyter case
    script_path = Path(__file__).resolve() if "__file__" in globals() else Path.cwd()
    print("Script path:", script_path)
    print("images/ exists:", Path("images").exists())
    if Path("images").exists():
        items = list(Path("images").iterdir())
        print("images/ contents:", [p.name for p in items])
    print("Trying to open:", IMG1_PATH, IMG2_PATH)

def generate_synthetic_images(p1: Path, p2: Path):
    print("Generating synthetic test images...")
    h, w = 600, 900
    base = np.full((h, w, 3), 255, dtype=np.uint8)
    cv2.rectangle(base, (80, 120), (700, 460), (10, 130, 200), -1)
    cv2.circle(base, (420, 300), 70, (200, 40, 30), -1)
    cv2.line(base, (50, 50), (850, 550), (0, 0, 0), 3)
    cv2.putText(base, "SIFT TEST", (120, 520), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 3, cv2.LINE_AA)
    cv2.imwrite(str(p1), base)
    M = cv2.getRotationMatrix2D((w//2, h//2), 25, 0.9)
    warped = cv2.warpAffine(base, M, (w, h), borderValue=(255, 255, 255))
    cv2.imwrite(str(p2), warped)
    print(f"Synthetic images saved: {p1}, {p2}")

def load_image_safe(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray, img

def create_sift():
    try:
        return cv2.SIFT_create()
    except Exception:
        try:
            return cv2.xfeatures2d.SIFT_create()
        except Exception:
            raise RuntimeError(
                "SIFT unavailable. Install with:\n"
                "pip install --upgrade opencv-contrib-python"
            )

def sift_detect_and_compute(sift, gray):
    return sift.detectAndCompute(gray, None)

def match_descriptors(des1, des2, ratio=RATIO_TEST):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    raw_matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for pair in raw_matches:
        if len(pair) == 2 and pair[0].distance < ratio * pair[1].distance:
            good.append(pair[0])
    return good

def compute_homography(kps1, kps2, matches):
    if len(matches) < 4:
        return None, None
    pts1 = np.float32([kps1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    pts2 = np.float32([kps2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, RANSAC_REPROJ_THRESH)
    return H, mask

def draw_matches_and_save(img1, img2, kps1, kps2, matches, mask=None, prefix="matches"):
    draw_all = cv2.drawMatches(img1, kps1, img2, kps2, matches[:MAX_MATCHES_DRAW], None,
                               flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    out_all = RESULTS_DIR / f"{prefix}_all.jpg"
    cv2.imwrite(str(out_all), draw_all)
    print("Saved:", out_all)

    if mask is not None:
        inlier_matches = [m for i, m in enumerate(matches) if mask[i]]
        draw_inliers = cv2.drawMatches(img1, kps1, img2, kps2, inlier_matches[:MAX_MATCHES_DRAW], None,
                                       flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        out_inliers = RESULTS_DIR / f"{prefix}_inliers.jpg"
        cv2.imwrite(str(out_inliers), draw_inliers)
        print("Saved:", out_inliers)
        return out_all, out_inliers
    return out_all, None

def warp_and_draw_outline(img1, img2, H):
    h1, w1 = img1.shape[:2]
    corners = np.float32([[0,0],[w1,0],[w1,h1],[0,h1]]).reshape(-1,1,2)
    warped = cv2.perspectiveTransform(corners, H)
    img2_copy = img2.copy()
    cv2.polylines(img2_copy, [np.int32(warped)], True, (0,255,0), 3, cv2.LINE_AA)
    out = RESULTS_DIR / "warped_outline.jpg"
    cv2.imwrite(str(out), img2_copy)
    print("Saved:", out)

def write_csv_summary(record, csv_path=RESULTS_DIR/"table_results.csv"):
    header = ["test", "kp1", "kp2", "good_matches", "inliers", "inlier_ratio"]
    exists = csv_path.exists()
    with open(str(csv_path), "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        if not exists:
            writer.writeheader()
        writer.writerow(record)
    print("Updated CSV summary:", csv_path)

def main():
    ensure_dirs()
    debug_print_paths()

    if not IMG1_PATH.exists() or not IMG2_PATH.exists():
        generate_synthetic_images(IMG1_PATH, IMG2_PATH)

    loaded1 = load_image_safe(IMG1_PATH)
    loaded2 = load_image_safe(IMG2_PATH)
    if not loaded1 or not loaded2:
        print("Could not load images. Check file integrity.")
        return

    g1, img1 = loaded1
    g2, img2 = loaded2

    try:
        sift = create_sift()
    except RuntimeError as e:
        print(e)
        return

    kps1, des1 = sift_detect_and_compute(sift, g1)
    kps2, des2 = sift_detect_and_compute(sift, g2)
    print(f"Keypoints: img1={len(kps1)}, img2={len(kps2)}")

    good = match_descriptors(des1, des2)
    print(f"Good matches after ratio test: {len(good)}")

    H, mask = compute_homography(kps1, kps2, good)
    inliers = int(np.sum(mask)) if mask is not None else 0
    print(f"Homography: {'Found' if H is not None else 'None'}, Inliers={inliers}")

    mask_flat = mask.ravel().tolist() if mask is not None else None
    draw_matches_and_save(img1, img2, kps1, kps2, good, mask_flat)

    if H is not None:
        warp_and_draw_outline(img1, img2, H)

    inlier_ratio = inliers / len(good) if good else 0
    record = {
        "test": "baseline",
        "kp1": len(kps1),
        "kp2": len(kps2),
        "good_matches": len(good),
        "inliers": inliers,
        "inlier_ratio": f"{inlier_ratio:.4f}",
    }
    write_csv_summary(record)

    print("\n✅ Done! Results saved in 'results/' folder.")

if __name__ == "__main__":
    main()


Working directory: /Users/anik/Desktop/cviu
Script path: /Users/anik/Desktop/cviu
images/ exists: True
images/ contents: ['.DS_Store']
Trying to open: images/img1.jpg images/img2.jpg
Generating synthetic test images...
Synthetic images saved: images/img1.jpg, images/img2.jpg
Keypoints: img1=152, img2=194
Good matches after ratio test: 75
Homography: Found, Inliers=67
Saved: results/matches_all.jpg
Saved: results/matches_inliers.jpg
Saved: results/warped_outline.jpg
Updated CSV summary: results/table_results.csv

✅ Done! Results saved in 'results/' folder.
